In [ ]:
#https://www.programmableweb.com/category/all/api-library?page=0


In [2]:
!pip install requests-random-user-agent
!pip install pandas
!pip install bs4
!pip install lxml

In [3]:
import multiprocessing
multiprocessing.cpu_count()

2

In [11]:

import requests
import requests_random_user_agent
#s = requests.Session()
#print(s.headers['User-Agent'])

# Without a session
resp = requests.get('https://httpbin.org/user-agent')
print(resp.json()['user-agent'])

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36


In [12]:
import concurrent.futures
import requests
import requests_random_user_agent
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

def _download_LIB(url):

  df_tempSDK = pd.DataFrame()

  rq = requests.get(url)
  
  if rq.status_code == 404 or rq.status_code == 403: ## Handle more error codes...
    exit

  main_data = rq.text

  # dataset from table
  df_tempSDK = pd.read_html(main_data, index_col=0)[0]

  main_soup = BeautifulSoup(main_data, 'html.parser')
  
  main_names = main_soup.find_all('tr')[1:245]

  list_urlSDK = []

  # Head url for meta_url
  head_Url= 'https://www.programmableweb.com'

  for row in main_names:
       text = row.find_all('td')[0]      
       list_urlSDK.append( (head_Url + str(text).partition('<a href="')[2].partition('">')[0])) # Meta URL 


  df_tempSDK['Meta_Url'] = list_urlSDK

  return df_tempSDK


def download_LIB(bulk_urls, num_Splits):
  df_temp = pd.DataFrame()

  # Split urls
  urls_splited  = np.array_split(bulk_urls, num_Splits) # max workers

  tasks = []

  for split in range(len(urls_splited)):
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(urls_splited)) as executor:
      for url in urls_splited[split]:    
        tasks.append(executor.submit(_download_LIB, url))
       ## mb call with bulk urls instead of one by one
  # Union
  for result in tasks:
    df_temp = df_temp.append(result.result())

  return df_temp


In [41]:
LIB_urls=[]
for i in range(10): ## web pages?.. 775+1
    main_url = 'https://www.programmableweb.com/category/all/api-library?page=' + str(i)
    LIB_urls.append(main_url)

In [42]:
#SDK_urls
df_LIB = pd.DataFrame()
df_LIB = download_LIB(LIB_urls, 10) # Num workers
#df_SDK.drop('Published', inplace=True, axis=1)
df_LIB

,Related APIs,Category,Published,Meta_Url
Library Name,,,,
GraphQL API for WordPress,NaN,GraphQL,02.16.2021,https://www.programmableweb.com/library/graphq...
DHTMLX JavaScript,NaN,Application Development,08.10.2020,https://www.programmableweb.com/library/dhtmlx...
Mocki NodeJS Library by Adalyte,NaN,Application Development,03.02.2020,https://www.programmableweb.com/library/mocki-...
Reddit.NET Library by Kris Craig,Reddit,Library,08.06.2019,https://www.programmableweb.com/library/reddit...
UNICORN Binance Websocket Library,Unicorn Binance Websocket,Financial,06.20.2019,https://www.programmableweb.com/library/unicor...
...,...,...,...,...
Duck Duck Go .NET Library by Tim Kelly,Duck Duck Go,Search,06.24.2015,https://www.programmableweb.com/library/duck-d...
Del.icio.us Java Library by David Czarnecki,Delicious,Bookmarks,06.24.2015,https://www.programmableweb.com/library/del.ic...
MyFitnessPal Go Library by Matt Ho,MyFitnessPal,Fitness,06.24.2015,https://www.programmableweb.com/library/myfitn...


In [43]:
# Creates new columns
df_LIB['Description'] = ""
df_LIB['Languages'] = ""
df_LIB['Related Frameworks'] = ""
df_LIB['Category'] = ""
df_LIB['Architectural Style'] = ""
df_LIB['Provider'] = ""
df_LIB['Asset URL'] = ""
df_LIB['Repository'] = ""
df_LIB['Terms Of Service'] = ""
df_LIB['Type'] = ""
df_LIB['Docs Home'] = ""
df_LIB['Request Formats'] = ""
df_LIB['Response Formats'] = ""

In [44]:
def _download_meta_url(df_source):

  for i in range(len(df_source)):

    meta_url = df_source['Meta_Url'][i]
    rq = requests.get(meta_url)

    if rq.status_code == 404 or rq.status_code == 403: ## Handle more error codes...
      continue

    meta_data = rq.text

    meta_soup = BeautifulSoup(meta_data, 'html.parser')

    # Update Description from the meta url
    meta_description = str(meta_soup.find('div', class_='tabs-header_description')).partition('">')[2].partition('</')[0]

    #print(meta_description)
    df_source['Description'][i] = meta_description 

    meta_specs = meta_soup.find('div', class_='section specs')

    #print(meta_specs)

    for lab in meta_specs.select("label"):   

      # Search for Related APIs
      if (lab.text.lower().find("related apis") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Related APIs'][i] =   lab.find_next_sibling().text

      # Search for Languages
      if (lab.text.lower().find("languages") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Languages'][i] =   lab.find_next_sibling().text

       # Search for Framework
      if (lab.text.lower().find("related framework") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Related Frameworks'][i] =   lab.find_next_sibling().text

       # Search for Categories and remplace them
      if (lab.text.lower().find("categories") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Category'][i] =   lab.find_next_sibling().text

      # Search for Architectural
      if (lab.text.lower().find("architectural style") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Architectural Style'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("library provider") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Provider'][i] =   lab.find_next_sibling().text

      # Search for URL
      if (lab.text.lower().find("asset home") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Asset URL'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("repository") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Repository'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("terms of service") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Terms Of Service'][i] =   lab.find_next_sibling().text
  
      # Search for Provider
      if (lab.text.lower().find("type") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Type'][i] =   lab.find_next_sibling().text

              
      # Search for Provider
      if (lab.text.lower().find("docs home") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Docs Home'][i] =   lab.find_next_sibling().text


      # Search for Provider
      if (lab.text.lower().find("request formats") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Request Formats'][i] =   lab.find_next_sibling().text


      # Search for Provider
      if (lab.text.lower().find("response formats") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Response Formats'][i] =   lab.find_next_sibling().text
  
    #print(df_source)

  return df_source

def download_meta_url(df_sourceCode, num_Splits):
  # Split dataframe
  dtframe_splited  = np.array_split(df_sourceCode, num_Splits) # max workers == num_Splits
  tasks = []

  for split in range(len(dtframe_splited)):
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(dtframe_splited)) as executor:
      # Download dataframes splited
      tasks.append(executor.submit(_download_meta_url, dtframe_splited[split]))

  df_temp = pd.DataFrame()
  # Union
  for result in tasks:
    df_temp = df_temp.append(result.result())

  return df_temp


In [45]:
df_LIB_metaData = pd.DataFrame()
df_LIB_metaData = download_meta_url(df_LIB, 10)
df_LIB_metaData

,Related APIs,Category,Published,Meta_Url,Description,Languages,Related Frameworks,Architectural Style,Provider,Asset URL,Repository,Terms Of Service,Type,Docs Home,Request Formats,Response Formats
Library Name,,,,,,,,,,,,,,,,
GraphQL API for WordPress,NaN,"GraphQL, Blogging, Open Source",02.16.2021,https://www.programmableweb.com/library/graphq...,GraphQL API for WordPress is an open source p...,PHP,,Indirect,Independent Developer,https://graphql-api.com/,https://github.com/leoloso/PoP/tree/master/lay...,,Product,,,
DHTMLX JavaScript,NaN,"Application Development, Developers",08.10.2020,https://www.programmableweb.com/library/dhtmlx...,DHTMLX offers a variety of JavaScript UI comp...,JavaScript,,Native/Browser,XB Software,https://dhtmlx.com/,https://github.com/DHTMLX,https://github.com/DHTMLX/gantt/blob/master/li...,Web/Internet,https://docs.dhtmlx.com/,,
Mocki NodeJS Library by Adalyte,NaN,"Application Development, API Design, API Strat...",03.02.2020,https://www.programmableweb.com/library/mocki-...,Mocki allows developers to create and setup m...,Node.js,,REST,,https://mocki.io,https://github.com/adalyte/mocki,,Web/Internet,https://mocki.io/docs,URI Query String/CRUD,JSON
Reddit.NET Library by Kris Craig,Reddit,"Library, Bots, Community, Content, Open Source...",08.06.2019,https://www.programmableweb.com/library/reddit...,Reddit.NET is a .NET Standard managed library...,".Net, C#",.NET,REST,Reddit,https://github.com/sirkris/Reddit.NET,https://github.com/sirkris/Reddit.NET,https://github.com/sirkris/Reddit.NET/blob/mas...,Web/Internet,https://github.com/sirkris/Reddit.NET/blob/mas...,"JSON, URI Query String/CRUD","JSON, Text"
UNICORN Binance Websocket Library,Unicorn Binance Websocket,"Financial, Analytics, Data",06.20.2019,https://www.programmableweb.com/library/unicor...,The UNICORN Binance WebSocket Library analyze...,Python,,Streaming,Unicorn Data Analysis,https://www.unicorn-data.com/unicorn-binance-w...,https://github.com/unicorn-data-analysis/unico...,https://github.com/unicorn-data-analysis/unico...,Unspecified,https://www.unicorn-data.com/unicorn-binance-w...,Websockets,JSON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Duck Duck Go .NET Library by Tim Kelly,Duck Duck Go,"Search, Privacy, Q&amp;A",06.24.2015,https://www.programmableweb.com/library/duck-d...,This is a .NET instant answer library written...,".Net, C#",,,Independent Developer,https://github.com/timkly/DuckDuckGo.Net,,,Library,,,
Del.icio.us Java Library by David Czarnecki,Delicious,"Bookmarks, Social",06.24.2015,https://www.programmableweb.com/library/del.ic...,The Del.icio.us Java library by David Czarnec...,Java,,,Independent Developer,https://github.com/czarneckid/delicious-java,,,Library,,,
MyFitnessPal Go Library by Matt Ho,MyFitnessPal,"Fitness, Food, Health, Sports",06.24.2015,https://www.programmableweb.com/library/myfitn...,The MyFitnessPal Go project by Matt Ho includ...,Go,,,Independent Developer,https://github.com/savaki/myfitnesspal,,,Library,,,


In [46]:
# Meta_Url could be used to check for updates on the source website. That uses only +600 web requests instead of +15k
# save a copy of the original dataframe to check for updates based on the meta url or other fields
df_LIB_metaData.reset_index(inplace=True)
df_LIB_metaData

,Library Name,Related APIs,Category,Published,Meta_Url,Description,Languages,Related Frameworks,Architectural Style,Provider,Asset URL,Repository,Terms Of Service,Type,Docs Home,Request Formats,Response Formats
0,GraphQL API for WordPress,NaN,"GraphQL, Blogging, Open Source",02.16.2021,https://www.programmableweb.com/library/graphq...,GraphQL API for WordPress is an open source p...,PHP,,Indirect,Independent Developer,https://graphql-api.com/,https://github.com/leoloso/PoP/tree/master/lay...,,Product,,,
1,DHTMLX JavaScript,NaN,"Application Development, Developers",08.10.2020,https://www.programmableweb.com/library/dhtmlx...,DHTMLX offers a variety of JavaScript UI comp...,JavaScript,,Native/Browser,XB Software,https://dhtmlx.com/,https://github.com/DHTMLX,https://github.com/DHTMLX/gantt/blob/master/li...,Web/Internet,https://docs.dhtmlx.com/,,
2,Mocki NodeJS Library by Adalyte,NaN,"Application Development, API Design, API Strat...",03.02.2020,https://www.programmableweb.com/library/mocki-...,Mocki allows developers to create and setup m...,Node.js,,REST,,https://mocki.io,https://github.com/adalyte/mocki,,Web/Internet,https://mocki.io/docs,URI Query String/CRUD,JSON
3,Reddit.NET Library by Kris Craig,Reddit,"Library, Bots, Community, Content, Open Source...",08.06.2019,https://www.programmableweb.com/library/reddit...,Reddit.NET is a .NET Standard managed library...,".Net, C#",.NET,REST,Reddit,https://github.com/sirkris/Reddit.NET,https://github.com/sirkris/Reddit.NET,https://github.com/sirkris/Reddit.NET/blob/mas...,Web/Internet,https://github.com/sirkris/Reddit.NET/blob/mas...,"JSON, URI Query String/CRUD","JSON, Text"
4,UNICORN Binance Websocket Library,Unicorn Binance Websocket,"Financial, Analytics, Data",06.20.2019,https://www.programmableweb.com/library/unicor...,The UNICORN Binance WebSocket Library analyze...,Python,,Streaming,Unicorn Data Analysis,https://www.unicorn-data.com/unicorn-binance-w...,https://github.com/unicorn-data-analysis/unico...,https://github.com/unicorn-data-analysis/unico...,Unspecified,https://www.unicorn-data.com/unicorn-binance-w...,Websockets,JSON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Duck Duck Go .NET Library by Tim Kelly,Duck Duck Go,"Search, Privacy, Q&amp;A",06.24.2015,https://www.programmableweb.com/library/duck-d...,This is a .NET instant answer library written...,".Net, C#",,,Independent Developer,https://github.com/timkly/DuckDuckGo.Net,,,Library,,,
246,Del.icio.us Java Library by David Czarnecki,Delicious,"Bookmarks, Social",06.24.2015,https://www.programmableweb.com/library/del.ic...,The Del.icio.us Java library by David Czarnec...,Java,,,Independent Developer,https://github.com/czarneckid/delicious-java,,,Library,,,
247,MyFitnessPal Go Library by Matt Ho,MyFitnessPal,"Fitness, Food, Health, Sports",06.24.2015,https://www.programmableweb.com/library/myfitn...,The MyFitnessPal Go project by Matt Ho includ...,Go,,,Independent Developer,https://github.com/savaki/myfitnesspal,,,Library,,,
248,Groupon Ruby Library by Wynn Netherland,Groupon,"Coupons, eCommerce",06.24.2015,https://www.programmableweb.com/library/groupo...,This is a Ruby wrapper for the Groupon API. I...,Ruby,,,Independent Developer,https://github.com/pengwynn/groupon,,,Library,,,


In [47]:
from datetime import datetime
datetime = datetime.now()

## Export the original + Meta_Url
# To CSV (index True 0,1,2...)
df_LIB_metaData.to_csv(r'/content/DataFrames/LIB_' + datetime.now().strftime('%d_%m_%Y') + '.csv', index = True, header = True)

# To JSON (columns format index True 0,1,2...)
df_LIB_metaData.to_json(r'/content/DataFrames/LIB_' + datetime.now().strftime('%d_%m_%Y') + '.json')
